In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import joblib

# Load the dataset
df = pd.read_csv('world_bank_open.csv')

# Drop columns with a high percentage of missing values
# Removed 'projectdoc', 'majorsector_percent', and 'theme' as they are not present in the DataFrame
df = df.drop(columns=[
    'mjsector1', 'mjsector2', 'mjsector3',
    'mjsector4', 'mjsector5', 'mjtheme1name', 'mjtheme2name', 'mjtheme3name',
    'mjtheme4name', 'mjtheme5name', 'Unnamed: 56', 'Country'
])

# Handle missing values
df = df.dropna(subset=['lendprojectcost', 'ibrdcommamt', 'idacommamt', 'totalamt', 'grantamt'])

# Convert columns with financial data to numeric types
financial_cols = ['lendprojectcost', 'ibrdcommamt', 'idacommamt', 'totalamt', 'grantamt']
df[financial_cols] = df[financial_cols].replace('[\$,]', '', regex=True).astype(float)

# Fill missing values in categorical columns with 'Unknown'
categorical_cols = ['lendinginstr', 'lendinginstrtype', 'envassesmentcategorycode', 'supplementprojectflg',
                    'productlinetype', 'projectstatusdisplay', 'status', 'borrower', 'impagency', 'sector',
                    'mjsector', 'goal', 'financier', 'location']
df[categorical_cols] = df[categorical_cols].fillna('Unknown')

# Convert categorical variables to one-hot encoding
categorical_features = categorical_cols

# Include regionname in the categorical features
categorical_features = categorical_cols + ['regionname']

preprocessor = ColumnTransformer(transformers=[
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
], remainder='drop') # Drop any remaining columns not specified in the transformer

# Define features and target variable
X = df.drop(columns=['id', 'project_name', 'url', 'boardapprovaldate', 'board_approval_month', 'closingdate'])
y = df['totalamt']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocess the features using the preprocessor pipeline
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

# Standardize the features
scaler = StandardScaler(with_mean=False) # Set with_mean=False to disable centering
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [2]:
# Train Linear Regression Model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Train Gradient Boosting Regression Model
gb_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
gb_model.fit(X_train, y_train)

# Save the models
joblib.dump(lr_model, 'linear_regression_model.pkl')
joblib.dump(gb_model, 'gradient_boosting_regression_model.pkl')


['gradient_boosting_regression_model.pkl']

In [5]:
# Define GAN components
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, output_dim)
        )

    def forward(self, x):
        return self.model(x)

class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# Hyperparameters
input_dim = X_train.shape[1]
output_dim = 1
lr = 0.0002
num_epochs = 40

# Initialize models and move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generator = Generator(input_dim, output_dim).to(device)
discriminator = Discriminator(input_dim + output_dim).to(device)

# Optimizers
optimizer_G = optim.Adam(generator.parameters(), lr=lr)
optimizer_D = optim.Adam(discriminator.parameters(), lr=lr)

# Loss function
criterion = nn.BCELoss()

# Prepare data for GAN and move to GPU
X_train_tensor = torch.tensor(X_train.toarray(), dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).to(device)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Training loop for GAN
for epoch in range(num_epochs):
    for real_data, real_labels in train_loader:
        batch_size = real_data.size(0)

        # Move real data and labels to GPU
        real_data = real_data.to(device)
        real_labels = real_labels.view(-1, 1).to(device)

        # Real data
        real_input = torch.cat((real_data, real_labels), dim=1)
        real_target = torch.ones(batch_size, 1).to(device)

        # Fake data
        noise = torch.randn(batch_size, input_dim).to(device)
        fake_labels = generator(noise).detach()
        fake_input = torch.cat((noise, fake_labels), dim=1)
        fake_target = torch.zeros(batch_size, 1).to(device)

        # Train Discriminator
        optimizer_D.zero_grad()
        real_loss = criterion(discriminator(real_input), real_target)
        fake_loss = criterion(discriminator(fake_input), fake_target)
        d_loss = real_loss + fake_loss
        d_loss.backward()
        optimizer_D.step()

        # Train Generator
        optimizer_G.zero_grad()
        generated_labels = generator(noise)
        g_input = torch.cat((noise, generated_labels), dim=1)
        g_loss = criterion(discriminator(g_input), real_target)
        g_loss.backward()
        optimizer_G.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, D Loss: {d_loss.item()}, G Loss: {g_loss.item()}')

# Save the GAN models
torch.save(generator.state_dict(), 'gan_generator.pth')
torch.save(discriminator.state_dict(), 'gan_discriminator.pth')

# Generate synthetic data
with torch.no_grad():
    synthetic_noise = torch.randn(X_train.shape[0], input_dim).to(device)
    synthetic_labels = generator(synthetic_noise).cpu().numpy()  # Move back to CPU for further processing
    synthetic_data = np.hstack((synthetic_noise.cpu().numpy(), synthetic_labels))

# Augment original data with synthetic data
augmented_X_train = np.vstack((X_train.toarray(), synthetic_data[:, :-1]))
augmented_y_train = np.hstack((y_train, synthetic_data[:, -1]))


Epoch 0, D Loss: 81.91875457763672, G Loss: 6.274112224578857
Epoch 10, D Loss: 83.6463623046875, G Loss: 28.164180755615234
Epoch 20, D Loss: 85.45469665527344, G Loss: 46.66402816772461
Epoch 30, D Loss: 94.54573059082031, G Loss: 36.662559509277344
